In [ ]:
# 準備工作

from bs4 import BeautifulSoup
# import lxml
import time
import datetime
import re
import sqlite3
import os
import json
import pprint

db_name = "../data/pacific.sqlite"
json_name = "../data/pacific.json"

In [ ]:
# 從db_name取出資料

with sqlite3.connect(db_name) as conn_retrieve:
    
    c_rt = conn_retrieve.cursor()
    qryString = "SELECT * from rental;"
    c_rt.execute(qryString)
    retrieved_list = c_rt.fetchall()

c_rt.close()

In [ ]:
# pprint.pprint((retrieved_list[1][4]))

# 定義方法

In [ ]:
# 引用 pacific_analysis_soup.py

# %load ../../Ivan/pacific_analysis_soup.py
"""
purpose:太平洋房屋 html檔案解析

input: html格式

output:dict檔案 ， 第二個參數若輸入True 則回傳json檔

ex: soup_analysis(soup) 

soup來源：DB[pacific.sqlite] → table[rental] → column[soup]

"""

def soup_analysis(soup,jsonOn=False):  #第二個參數輸入True 格式回傳json檔
    
    in_key = {} #所有的內部真實儲存欄位

    #標題
    in_key['標題'] = soup.find('h1','ng-binding').text[:-6] #去掉後面編號

    #編號
    in_key['編號'] = soup.find('h1','ng-binding').text[-6:] #去掉後面編號
    
    #地址
    in_key['地址'] = soup.find('h2','ng-binding').text

    #價格
    in_key['價格'] = soup.find('b','ng-binding').text

    #細項 [格局一]#
    watch_1 = soup.find_all('li','ng-binding')


    #坪數、押金、格局、樓層/樓高、出租型態、可遷入日期

    for watch in watch_1:
        key_and_value = watch.text.split('：')
        in_key[key_and_value[0].replace('  ','')] = key_and_value[1]


    # 車位、公車站名、管理費/收取方式、鄰近捷運、朝向、鄰近公園、
    # 屋齡、鄰近市場、租賃期限、鄰近學校、房屋特色

    watch_2 = soup.find('table','section_table')
    watch_2_keys = watch_2.find_all('th') #找出key值
    watch_2_values = watch_2.find_all('td') #找出list值

    #清洗key值並塞進 key_lists
    key_lists = []     
    for a in watch_2_keys:
        key_lists.append(a.text.replace(' ',''))

    #用迴圈去跑，一個key值加入一個values值
    number = 0          
    for a in watch_2_values:
        in_key[key_lists[number]] = re.sub(" |\n|\t",'',a.text)
        number = number + 1  


    #出租說明、公共設施、居家設備、傢俱
    
    watch_3 = soup.find('table','section_table rent_eq')
    watch_3 = watch_3.find_all('tr')

    #在watch_3裡面做迴圈，把打勾的加進字串
    for every_watch_3 in watch_3:
        category_name = every_watch_3.find('th').text.replace(
            '\u3000','')
        checked_set = every_watch_3.find_all(
            'div',class_=re.compile('with$'))
        checked_set = [re.sub(' |\n','',each.text) 
                       for each in checked_set]
        
        # 把每個迴圈的title拉出來並塞進字串
        in_key[category_name] = checked_set  
        

    #經緯度
    
    lati_longi = soup.find('a','bbtn bbtn_g bbtn_vr')[
        'href'].split('?')[1].replace('lat=','').replace(
        'lng=','').replace('&',',').split(',')
    in_key['經度'] = lati_longi[0]
    in_key['緯度'] = lati_longi[1]

    
#     print(type(in_key))
    
    if jsonOn == False:
        return in_key
    else:
        return json.dumps(in_key,ensure_ascii=False)

# 執行程序

In [ ]:
# 迴圈1
# 將所有的內頁資料清理並存成dictionary

dict_all = {}

for each in retrieved_list:
    
    input_soup = each[1]
#     input_soup = BeautifulSoup(input_soup,'lxml')
    input_soup = BeautifulSoup(input_soup,'html.parser')
    
    # 用if避開空內頁
    check_title = input_soup.find("title")
    
    if check_title != None:
        
        if check_title.text != "太平洋房屋":
        
            url = each[0]
            page_id = url.split("=")[-1]
            output = soup_analysis(input_soup,jsonOn=False)
            output['爬蟲日期時間'] = each[4]
            dict_all[page_id] = output

In [ ]:
# 檢查結果

# len(dict_all)
# pprint.pprint(dict_all)

In [ ]:
# 將最後的結果 dict_all 以json格式存到 json_name 路徑裡

if os.path.isfile(json_name) == False:
    json = json.dumps(dict_all,ensure_ascii=False)
    f = open(json_name,"w")
    f.write(json)
    f.close()

In [ ]:
# 檢查結果

# if os.path.isfile(json_name) == True:
#     with open(json_name) as json_file:  
#         data = json.load(json_file)
#         pprint.pprint(data)
# else:
#     print("檔案不在路徑 " + str(json_name) + " 中")